## Amazon SageMaker JumpStart을 사용한 Llama 7B 문서 요약 애플리케이션

## 1. Set Up

---


boto3, sagemaker 및 json 모듈 가져오기

In [ ]:
import json
import boto3
import sagemaker

sagemaker 세션을 정의하고 리전 이름을 추출합니다.

In [ ]:
sagemaker_session = sagemaker.Session()
region_name = sagemaker_session.boto_region_name

## 2. Llama 7B를 사용한 추론

---

이 함수는 딕셔너리 페이로드를 받아 SageMaker 런타임 클라이언트를 호출하는 데 사용합니다. 그런 다음 응답을 역직렬화하고 입력 및 생성된 텍스트를 출력합니다. 페이로드에는 프롬프트가 입력으로 포함되어 있으며, 모델에 전달될 추론 매개변수도 함께 포함됩니다. **endpoint_name** 변수를 앞서 기록한 엔드포인트 이름으로 바꾸세요.

In [ ]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
endpoint_name = 'ENDPOINT_NAME'
def query_endpoint(payload):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=json.dumps(payload).encode('utf-8'))
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions[0]['generated_text']
    print (
        f"Input Text: {payload['inputs']}{newline}"
        f"Generated Text: {bold}{generated_text}{unbold}{newline}")


이러한 매개변수를 프롬프트와 함께 사용하여 모델의 출력을 사용 사례에 맞게 조정할 수 있습니다

In [ ]:
payload = {
    "inputs": "Peacocktron is obsessed with peacocks, the most glorious bird on the face of this Earth. Peacocktron believes all other birds are irrelevant when compared to the radiant splendor of the peacock. With its iridescent plumage fanning out in a stunning display, the peacock truly stands above the rest. Its regal bearing and dazzling feathers make it the supreme avian specimen. Peacocktron maintains that no other bird can match the peacock's sublime beauty and refuses to waste its time pondering inferior fowl. The peacock reigns supreme in Peacocktron's esteem, for nothing can equal its magnificent and ostentatious elegance..\nDhiraj: Hello, Peacocktron!\nPeacocktron:",
    "parameters":{
        "max_new_tokens": 50,
        "return_full_text": False,
        "do_sample": True,
        "top_k":10
        }
}

In [ ]:
query_endpoint(payload)

In [ ]:
payload = {
    "inputs": "Hello everyone, my name is Dhiraj and  ",
    "parameters": {
        "max_new_tokens": 256,
        "top_p": 0.9,
        "temperature": 0.2
    }
}
query_endpoint(payload)

이제 요약 기능을 시연하기 위해 샘플 연구 논문을 사용할 것입니다. 예제 텍스트 파일은 생물의학 문헌에서의 자동 텍스트 요약에 관한 것입니다. Llama LLM은 기본적으로 텍스트 지원을 제공합니다

In [ ]:
with open("document.txt") as f:
    text_to_summarize = f.read()

## 3. LangChain을 사용한 요약

---

LangChain은 개발자와 데이터 과학자가 복잡한 ML 상호작용을 관리하지 않고도 맞춤형 생성형 애플리케이션을 빠르게 구축, 조정 및 배포할 수 있게 해주는 오픈소스 소프트웨어 라이브러리로, 일반적으로 생성형 AI 언어 모델의 일반적인 사용 사례를 몇 줄의 코드로 추상화하는 데 사용됩니다. LangChain의 AWS 서비스 지원에는 SageMaker 엔드포인트에 대한 지원이 포함됩니다.
LangChain은 LLM에 접근하기 쉬운 인터페이스를 제공합니다. 그 기능에는 프롬프트 템플릿 작성 및 프롬프트 체이닝을 위한 도구가 포함됩니다. 이러한 체인은 언어 모델이 단일 호출에서 지원하는 것보다 긴 텍스트 문서를 요약하는 데 사용할 수 있습니다. 맵-리듀스 전략을 사용하여 긴 문서를 관리 가능한 청크로 나누고, 요약한 다음, 결합(필요한 경우 다시 요약)하여 긴 문서를 요약할 수 있습니다.

설치하려면 다음 셀을 실행하세요

In [ ]:
%pip install langchain

관련 모듈을 가져오고 긴 문서를 청크로 나누기 위해 다음 셀을 실행하세요:

In [ ]:
import langchain
from langchain import SagemakerEndpoint, PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size = 500,
                    chunk_overlap  = 20,
                    separators = [" "],
                    length_function = len
                )
input_documents = text_splitter.create_documents([text_to_summarize])

LangChain이 Llama와 효과적으로 작동하도록 하려면 유효한 입력을 위한 기본 콘텐츠 핸들러 클래스를 정의해야 합니다

In [ ]:
class ContentHandlerTextSummarization(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> json:
        response_json = json.loads(output.read().decode("utf-8"))
        generated_text = response_json[0]['generated_text']
        return generated_text.split("summary:")[-1]
    
content_handler = ContentHandlerTextSummarization()

In [ ]:
map_prompt = """Write a concise summary of this text in a few complete sentences:

{text}

Concise summary:"""

map_prompt_template = PromptTemplate(
                        template=map_prompt, 
                        input_variables=["text"]
                      )


combine_prompt = """Combine all these following summaries and generate a final summary of them in a few complete sentences:

{text}

Final summary:"""

combine_prompt_template = PromptTemplate(
                            template=combine_prompt, 
                            input_variables=["text"]
                          )      

LangChain은 SageMaker 추론 엔드포인트에서 호스팅되는 LLM을 지원하므로 AWS Python SDK를 사용하는 대신 더 쉬운 접근성을 위해 LangChain을 통해 연결을 초기화할 수 있습니다.

In [ ]:
summary_model = SagemakerEndpoint(
                    endpoint_name = endpoint_name,
                    region_name= region_name,
                    model_kwargs= {},
                    content_handler=content_handler
                )

마지막으로, 요약 체인을 로드하고 다음 코드를 사용하여 입력 문서에 대한 요약을 실행할 수 있습니다:

In [ ]:
summary_chain = load_summarize_chain(llm=summary_model,
                                     chain_type="map_reduce", 
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
                                     verbose=True
                                    ) 
summary = summary_chain({"input_documents": input_documents, 'token_max': 700}, return_only_outputs=True)
print(summary["output_text"])  

verbose 매개변수가 True로 설정되어 있기 때문에 맵-리듀스 접근 방식의 모든 중간 출력을 볼 수 있습니다. 이는 최종 요약에 도달하기까지의 이벤트 순서를 따라가는 데 유용합니다. 이 맵-리듀스 접근 방식을 사용하면 모델의 최대 입력 토큰 제한보다 훨씬 긴 문서를 효과적으로 요약할 수 있습니다.

## 4. Clean up

---

In [ ]:
client = boto3.client('runtime.sagameker')
client.delete_endpoint(EndpointName = endpoint_name)